# this brings together all we learned today

Using variable `tas`, mask over land, for `historical` for the 1980-2010 period
Show spatial difference map from the time standard deviation of departures from a 1970-2000 climatolgy for the each model ensemble average vs your choice of temperature obs over the same periods. 
Do this for all models and plot them all on one plot, with the upper/left plot showing the obs standard_deviation.
Use latest version if multiple are present

**Bonus:** Produce the median model

# Step 1: Locate Models files

In [1]:
import genutil
import glob

template = '%(root)/%(collection)/%(type)/%(institution)/%(model)/%(experiment)/%(member)/%(table)/%(variable)/%(grid)/%(version)/%(variable)_%(table)_%(model)_%(experiment)_%(member)_%(grid)_%(period).nc'
path = genutil.StringConstructor(template)
for key in path.keys():
    setattr(path, key, "*")

path.root = '/global/cscratch1/sd/cmip6'  # this allows to easily change this from one machine to another
path.collection = 'CMIP6'
path.type = 'CMIP'
path.experiment = "historical"
path.table = "Amon"
path.variable = "tas"
print("path:", path())
all_files = glob.glob(path())
models = set()

for fname in all_files:
    sp = fname.split("/")
    models.add(sp[8])
print("models:", models)

path: /global/cscratch1/sd/cmip6/CMIP6/CMIP/*/*/historical/*/Amon/tas/*/*/tas_Amon_*_historical_*_*_*.nc
models: {'GISS-E2-1-H', 'GISS-E2-1-G', 'MIROC6', 'GFDL-ESM4', 'SAM0-UNICON', 'CanESM5', 'CAMS-CSM1-0', 'UKESM1-0-LL', 'EC-Earth3-Veg', 'BCC-ESM1', 'CESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'BCC-CSM2-MR', 'CNRM-ESM2-1', 'CNRM-CM6-1', 'GFDL-CM4', 'HadGEM3-GC31-LL', 'MRI-ESM2-0', 'CESM2-WACCM'}


In [2]:
# Step 2 generate xml files
from subprocess import PIPE, Popen
import cdutil
import cdms2
import os

cdms2.setCompressionWarnings(0)
start = '1980'
end = '2010'

climo_start = '1970'
climo_end = '2000'
xml_name_template = "%(variable)_%(model)_%(member).%(extension)"
xml_name = genutil.StringConstructor(xml_name_template)
xml_name.variable = path.variable
for model in models:
    print("Processing:", model)
    xml_name.model = model
    xml_name.member = "avg"
    xml_name.extension = "nc"
    if os.path.exists(xml_name()):
        print("Already here, skipping")
        continue
    xml_name.extension = "xml"
    path.version = "*"
    path.member = "*"
    path.model = model
    # figure out versions:
    model_files = glob.glob(path())
    versions = set()
    members = set()
    for name in model_files:
        sp = name.split("/")
        versions.add(sp[-2])
        members.add(sp[-6])
    # print("\tversions:", sorted(versions), sorted(members))
    path.version = sorted(versions)[-1]  # latest version
    departures = None
    nUsed = 0
    for member in members:
        #if nUsed > 3:
        #    break
        path.member = member
        files = glob.glob(path())
        if len(files)>0:  # this member has data
            print("\tProcessing:", member, len(files))
            nUsed += 1
            xml_name.member = member
            if len(files)>1:
                cmd = "cdscan -x {} {}".format(xml_name(), " ".join(files))
                #print("\tCDSCAN COMMNAD:", cmd)
                p = Popen(cmd.split(), stderr=PIPE, stdout=PIPE)
                o, e = p.communicate()
                #print("Output:", o.decode())
                #input("PRess enter")
                xml_name.extension = "xml"
                f = cdms2.open(xml_name())
            else:
                f = cdms2.open(files[0])
            data = f(path.variable, time=(start, end, "con"))
            f.close()
            if len(files)>1:
                os.remove(xml_name())
            climo = cdutil.ANNUALCYCLE.climatology(data(time=(climo_start, climo_end, "con")))
            if departures is None:
                departures = cdutil.ANNUALCYCLE.departures(data, ref=climo)
            else:
                departures += cdutil.ANNUALCYCLE.departures(data, ref=climo)
    departures /= nUsed
    xml_name.extension = "nc"
    xml_name.member = "avg"
    out = xml_name()
    print("\t\tWriting {} to {}".format(out, departures.shape))
    import importlib
    importlib.reload(cdms2)
    fo = cdms2.open(out,'w')
    fo.write(departures, id=path.variable)
    fo.close()

Processing: GISS-E2-1-H
Already here, skipping
Processing: GISS-E2-1-G
Already here, skipping
Processing: MIROC6
Already here, skipping
Processing: GFDL-ESM4
Already here, skipping
Processing: SAM0-UNICON
Already here, skipping
Processing: CanESM5
Already here, skipping
Processing: CAMS-CSM1-0
Already here, skipping
Processing: UKESM1-0-LL
Already here, skipping
Processing: EC-Earth3-Veg
Already here, skipping
Processing: BCC-ESM1
Already here, skipping
Processing: CESM2
Already here, skipping
Processing: IPSL-CM6A-LR
Already here, skipping
Processing: EC-Earth3
Already here, skipping
Processing: BCC-CSM2-MR
Already here, skipping
Processing: CNRM-ESM2-1
Already here, skipping
Processing: CNRM-CM6-1
Already here, skipping
Processing: GFDL-CM4
Already here, skipping
Processing: HadGEM3-GC31-LL
Already here, skipping
Processing: MRI-ESM2-0
Already here, skipping
Processing: CESM2-WACCM
Already here, skipping


Now let's do the same thing for the obs

In [3]:
obs_path = '/global/cscratch1/sd/cmip6/MERRA2/CREATE-IP.reanalysis.NASA-GMAO.MERRA-2.atmos.mon/tas_Amon_reanalysis_MERRA2_198001-201902.nc'
f = cdms2.open(obs_path)
tas = f("tas", time=(start, end, "con"))
climo = cdutil.ANNUALCYCLE.climatology(tas(time=(climo_start, climo_end, "con")))
departures = cdutil.ANNUALCYCLE.departures(tas, ref=climo)
f = cdms2.open("tas_reanalysis_MERRA2.nc", "w")
f.write(departures, id="tas")

In [4]:
n = len(models)
print("N Models:", n)

N Models: 20


With obs it's 21 models, let's plot this in portrait mode with 3 columns

In [5]:
import EzTemplate
import vcs

canvas = vcs.init()
template = vcs.createtemplate()
template.blank()
template.legend.priority = 1
template.data.priority = 1
template.dataname.priority=1

Multi = EzTemplate.Multi(rows=7, columns=3, x=canvas, template=template)

Let's plot the obs

In [6]:
canvas.clear()
target_grid = cdms2.createGaussianGrid(64)
import genutil
std_obs = genutil.statistics.std(departures, axis="t")
std_obs = std_obs.regrid(target_grid)(latitude=(-90,90))  # Flip it back
std_obs.id = "MERRA2"
tmpl = Multi.get(row=0, column=0)
tmpl.legend.priority=0  # turn off legend
canvas.plot(std_obs, tmpl, ratio="autot")

/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1313: Warning: 
avariable.regrid: We chose regridTool = esmf for you among the following choices:
   Tools ->    'regrid2' (old behavior)
               'esmf' (conserve, patch, linear) or
               'libcf' (linear)
  warnings.warn(message, Warning)
/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1320: Warning: 
avariable.regrid: We chose regridMethod = linear for you among the following choices:
    'conserve' or 'linear' or 'patch'
  warnings.warn(message, Warning)


In [7]:
column = 1
row = 0
xml_name.member = "avg"
xml_name.extension = "nc"
obs_grid = std_obs.getGrid()
isof = vcs.createisofill()
levels = vcs.mkscale(-2.5,2.5)
isof.levels = levels
isof.ext_1 = True
isof.ext_2 = True
isof.fillareacolors = vcs.getcolors(isof.levels)
for model in models:
    print("Processing:", model)
    xml_name.model = model
    f = cdms2.open(xml_name())
    tas = f("tas")
    print("\t\t\t", tas.shape)
    std = genutil.statistics.std(tas, axis='t')
    diff = std.regrid(obs_grid) - std_obs
    diff.id = model
    print("\t\t\t", genutil.minmax(diff))
    tmpl = Multi.get(row=row, column=column)
    if column==1 and row==0:
        tmpl.legend.priority=1  # turn on legend for this guy
    d = canvas.plot(diff, tmpl, isof, ratio="autot")
    column += 1
    if column == 3:
        column=0
        row+=1
d

Processing: GISS-E2-1-H
			 (360, 90, 144)
			 (-3.0522962606167208, 1.62485059206849)
Processing: GISS-E2-1-G
			 (30, 90, 144)
			 (-3.530624381875452, 2.019747920002701)
Processing: MIROC6
			 (360, 128, 256)
			 (-3.2088803217043367, 0.15972578181515784)
Processing: GFDL-ESM4
			 (360, 180, 288)
			 (-1.3458459762030206, 1.9394532287909723)
Processing: SAM0-UNICON
			 (360, 192, 288)
			 (-1.3087181165501227, 3.1023687487029603)
Processing: CanESM5
			 (360, 64, 128)
			 (-3.6481493665047084, 3.2091581553708703)
Processing: CAMS-CSM1-0
			 (360, 160, 320)
			 (-1.8709863755895317, 2.6298425785273207)
Processing: UKESM1-0-LL
			 (360, 144, 192)
			 (-0.9841153882248268, 3.481194155245584)
Processing: EC-Earth3-Veg
			 (360, 256, 512)
			 (-1.040407354975554, 2.354552863610547)
Processing: BCC-ESM1
			 (360, 64, 128)
			 (-2.310425046647179, 2.1646843427385587)
Processing: CESM2
			 (360, 192, 288)
			 (-1.2980839748509672, 2.0270577388772133)
Processing: IPSL-CM6A-LR
			 (360, 143, 

In [8]:
canvas.png("final")

![](final.png)